## Georelation explained

In [1]:
import lxml.etree as etree
import requests
import cioppy

import numpy as np
import ipyleaflet
from shapely.geometry import box
from shapely.geometry import multipolygon
from shapely.wkt import loads
from ipyleaflet import Map, Polygon

ciop = cioppy.Cioppy()

Define the Sentinel-1 endpoint

In [2]:
s1_osd_url = 'https://catalog.terradue.com/sentinel1/description'

Define an area of interest with a polygon over Greece expressed as a Well-Known Text:


In [3]:
aoi_wkt = 'POLYGON ((20.9113 39.4866, 20.9113 40.0866, 20.3113 40.0866, 20.3113 39.4866, 20.9113 39.4866))'

In [4]:
search_params = dict([('count', '2'),
                      ('geom', aoi_wkt)])

In [5]:
search = ciop.search(end_point=s1_osd_url,
                     params=search_params,
                     output_fields='self,productType,track,enclosure,identifier,wkt,startdate', 
                     model='EOP')

In [6]:
search

[{'enclosure': 'https://store.terradue.com/download/sentinel1/files/v1/S1A_IW_RAW__0SDV_20181002T043940_20181002T044012_023952_029D99_1926',
  'identifier': 'S1A_IW_RAW__0SDV_20181002T043940_20181002T044012_023952_029D99_1926',
  'productType': 'RAW',
  'self': 'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_RAW__0SDV_20181002T043940_20181002T044012_023952_029D99_1926',
  'startdate': '2018-10-02T04:39:40.4760000Z',
  'track': '80',
  'wkt': 'POLYGON((20.0101 39.6281,19.6112 37.6733,22.378 37.403,22.8542 39.3563,20.0101 39.6281))'},
 {'enclosure': 'https://store.terradue.com/download/sentinel1/files/v1/S1A_IW_SLC__1SDV_20181002T043943_20181002T044010_023952_029D99_07CD',
  'identifier': 'S1A_IW_SLC__1SDV_20181002T043943_20181002T044010_023952_029D99_07CD',
  'productType': 'SLC',
  'self': 'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20181002T043943_20181002T044010_023952_029D99_07CD',
  'startdate': '2018-10-02T04:39:43.1980000Z

The search results provide the product footprint as a Well-Known Text (WKT) Polygon:

In [7]:
search[0]['wkt']

'POLYGON((20.0101 39.6281,19.6112 37.6733,22.378 37.403,22.8542 39.3563,20.0101 39.6281))'

This WKT can be loaded and asked for the exterior coordinates as an array:

In [8]:
loads(search[0]['wkt']).exterior.coords[:]

[(20.0101, 39.6281),
 (19.6112, 37.6733),
 (22.378, 37.403),
 (22.8542, 39.3563),
 (20.0101, 39.6281)]

ipyleaflet expects the coordinates in the lat, lon order. Numpy will swap the order of the values with:

In [9]:
np.asarray([t[::-1] for t in list(loads(search[0]['wkt']).exterior.coords)])

array([[ 39.6281,  20.0101],
       [ 37.6733,  19.6112],
       [ 37.403 ,  22.378 ],
       [ 39.3563,  22.8542],
       [ 39.6281,  20.0101]])

In [10]:
np.asarray([t[::-1] for t in list(loads(search[0]['wkt']).exterior.coords)]).tolist()

[[39.6281, 20.0101],
 [37.6733, 19.6112],
 [37.403, 22.378],
 [39.3563, 22.8542],
 [39.6281, 20.0101]]

Furthermore, ipyleaflet expects a list so the complete transformation of the WKT expression to ipyleaflet is:

In [11]:
search_locations = np.asarray([t[::-1] for t in list(loads(search[0]['wkt']).exterior.coords)]).tolist()

Let's apply the same for our AOI:

In [12]:
aoi_locations = np.asarray([t[::-1] for t in list(loads(aoi_wkt).exterior.coords)]).tolist()

The map will be centered on the AOI centroid:

In [13]:
map_center_x = loads(aoi_wkt).centroid.x
map_center_y = loads(aoi_wkt).centroid.y

Now we'll create the map:

In [14]:
m = Map(center=(loads(aoi_wkt).centroid.y, 
                loads(aoi_wkt).centroid.x), 
                zoom=6)

for index, elem in enumerate(search):

    search_locations = np.asarray([t[::-1] for t in list(loads(elem['wkt']).exterior.coords)]).tolist()
    
    m += Polygon(locations=search_locations,
                 color="red",
                 fill_color="red",
                 weight=1,
                 fill_opacity=0.1)

m += Polygon(locations=aoi_locations,
             color="green",
             fill_color="green",
             weight=1,
             fill_opacity=0.1)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …